NOTE: This is largely pulled from QBias' datacrawl, AllsidesDataCrawl.ipynb, which they used to scrape links from a table and, subsequently, the associated content from AllSides' news' roundup through to winter of 2022 (https://github.com/irgroup/Qbias/blob/main/Scraper/AllsidesDataCrawl.ipynb)

Unfortunately, AllSides seems to have updated its site and the scraper (with Selenium) as written triggers a TimeoutException when attempting to scrape links from AllSides' complied table. Attempts to conditionally handle cookie popups and similar, change waittimes, update the pagination button class, and use a more generic Xpath for the table doesn't resolve this, as the scraper fails to gauge the correct number of pages.

An attempt to use BeautifulSoup didn't seem to work as well. Based on what I can gauge on the topic via StackOverflow, the implication seems to be that the table isn't in the static HTML, as JavaScript fetches & renders it after the page loads, blocking the content.

Consequently, *this file will not run successfully*; I've included it here to demonstrate the source of the 2022 data and document my attempts at creating a new scraper.


In [1]:
import pickle
import re
from tqdm import tqdm
import time
import numpy as np
from IPython.display import clear_output
import pandas as pd
import time

In [12]:
#!wget -O /tmp/chrome.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
#!dpkg -i /tmp/chrome.deb
#!apt-get -f install -y

--2025-03-08 05:29:00--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.251.189.190, 142.251.189.93, 142.251.189.91, ...
Connecting to dl.google.com (dl.google.com)|142.251.189.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114757600 (109M) [application/x-debian-package]
Saving to: ‘/tmp/chrome.deb’

/tmp/chrome.deb     100%[===================>] 109.44M   352MB/s    in 0.3s    

2025-03-08 05:29:00 (352 MB/s) - ‘/tmp/chrome.deb’ saved [114757600/114757600]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 125113 files and directories currently installed.)
Preparing to unpack /tmp/chrome.deb ...
Unpacking google-chrome-stable (134.0.6998.35-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Package libvulkan1 is not installed.

dpkg: error processing package google

In [19]:
# !apt-get update
# !apt-get install -y wget unzip
# !apt-get install -y google-chrome-stable

Get:1 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,211 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,036 B in 2s (1,824 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repos

In [20]:
# !google-chrome --version

Google Chrome 134.0.6998.35 


In [23]:
# Replace with the correct ChromeDriver link for your version!
chromedriver_url = "https://storage.googleapis.com/chrome-for-testing-public/134.0.6998.35/linux64/chromedriver-linux64.zip"

# Download and install ChromeDriver
!wget -q -O /tmp/chromedriver.zip {chromedriver_url}
!unzip -o /tmp/chromedriver.zip -d /tmp/
!chmod +x /tmp/chromedriver-linux64/chromedriver
!mv /tmp/chromedriver-linux64/chromedriver /usr/bin/chromedriver

# Verify the installation
!chromedriver --version


Archive:  /tmp/chromedriver.zip
  inflating: /tmp/chromedriver-linux64/LICENSE.chromedriver  
  inflating: /tmp/chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver  
  inflating: /tmp/chromedriver-linux64/chromedriver  
ChromeDriver 134.0.6998.35 (ea6ef4c2ac15ae95d2cfd65682da62c093415099-refs/branch-heads/6998@{#1472})


In [24]:
#!pip install selenium
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

In [25]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [30]:
# Set paths for Chrome and ChromeDriver
chrome_path = "/usr/bin/google-chrome"
chromedriver_path = "/usr/bin/chromedriver"

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run without GUI
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.binary_location = chrome_path  # Set correct Chrome binary

# Set up WebDriver
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Test if it works
driver.get("https://www.google.com")
print("Page title:", driver.title)

# Close the driver
#driver.quit()

Page title: Google


In [31]:
wait_time = 0.50

In [32]:
# chromedriver setup

serv = Service(r'driver/chromedriver') #path from 'which chromedriver'

chrome_options = Options()

chrome_options.add_argument("--headless")

In [ ]:
# assemble list of links to all articles
links = []

# interact with cookie terms
chrome_path = r'driver/chromedriver' #path from 'which chromedriver'
chrome_options = Options()
chrome_options.add_argument("--headless") # open chrome in background
driver = webdriver.Chrome(service=serv, options=chrome_options)
driver.get('https://www.allsides.com/headline-roundups')
wait = WebDriverWait(driver, 10)
wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'css-47sehv')))

ele = driver.find_element(By.CLASS_NAME, "css-47sehv")
ele.click()

# retireve number of pages
last_page_button = driver.find_element(By.CLASS_NAME, "pager-last")
link_last_page = last_page_button.find_elements(By.TAG_NAME, "a")
t = link_last_page[0].get_attribute("href")
last_page_index = int(t[-3:])

# retrieve links from start page
main_table = driver.find_element(By.XPATH, "//*[@id=\"block-views-de37fa32ea86f5545eb9b7722977a70d\"]/div/div[2]/table/tbody") # table body

rows = main_table.find_elements(By.TAG_NAME, "tr")
for i in rows:
    entry = i.find_elements(By.TAG_NAME, "td")
    link = entry[0].find_element(By.TAG_NAME, "a")
    links.append(link.get_attribute("href"))
WebDriverWait(driver, 20)

# retrieve links for other pages
for page in tqdm(range(2,last_page_index+1)):   # set to max number of pages
    driver.get("https://www.allsides.com/headline-roundups?page="+str(page))
    WebDriverWait(driver, 20)
    main_table = driver.find_element(By.XPATH, "//*[@id=\"block-views-de37fa32ea86f5545eb9b7722977a70d\"]/div/div[2]/table/tbody") # table body

    rows = main_table.find_elements(By.TAG_NAME, "tr")
    for i in rows:
        entry = i.find_elements(By.TAG_NAME, "td")
        link = entry[0].find_element(By.TAG_NAME, "a")
        links.append(link.get_attribute("href"))

driver.close()

In [ ]:
#Troubleshooting Link Retrieval: - CF

import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

#Initialize WebDriver
chrome_path = "/usr/bin/chromedriver"
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service(chrome_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

#Open Website
driver.get("https://www.allsides.com/headline-roundups")
wait = WebDriverWait(driver, 20)  #Increased timeout

#Force JavaScript to Render Page
def wait_for_js_load():
    time.sleep(3)
    for _ in range(10):  #Scroll multiple times to load content
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
        time.sleep(2)  #Allow content to load

    #Ensure the page is truly loaded before proceeding
    wait.until(lambda d: d.execute_script("return document.readyState") == "complete")
    time.sleep(3)  #Extra wait

wait_for_js_load()

#Save Screenshots with File Existence Check
screenshot_path_before = "/content/debug_before.png"
screenshot_path_after = "/content/debug_after.png"

driver.save_screenshot(screenshot_path_before)
if os.path.exists(screenshot_path_before):
    print(f"Screenshot successfully saved: {screenshot_path_before}")
else:
    print("Screenshot saving failed!")

#Wait for Table to Appear Before Screenshot
try:
    print("Waiting for table...")
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.views-table.cols-3")))
    print("Table found! Taking another screenshot...")

    driver.save_screenshot(screenshot_path_after)
    if os.path.exists(screenshot_path_after):
        print(f"Screenshot successfully saved: {screenshot_path_after}")
    else:
        print("Screenshot saving failed!")
except:
    print("Table STILL not found after waiting!")

#Close WebDriver
driver.quit()

#List Files Again
print("\n Checking saved files in /content/:")
os.system("ls -lh /content/")


In [ ]:
#Alternative: BeautifulSoup -CF

import requests
from bs4 import BeautifulSoup

base_url = "https://www.allsides.com/headline-roundups"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36"
}

response = requests.get(base_url, headers=headers)

soup = BeautifulSoup(response.text, "html.parser")

table = soup.find("table", class_="views-table cols-3")

if table:
    print("Table found, Extracting links...")
    links = []

    for row in table.find_all("tr"):
        link_tag = row.find("td", class_="views-field views-field-name")
        if link_tag:
            a_tag = link_tag.find("a")
            if a_tag and "href" in a_tag.attrs:
                links.append("https://www.allsides.com" + a_tag["href"])

    print(f"Scraped {len(links)} article links:")
    print(links[:5])
else:
    print("Table not found! JavaScript might be blocking the content.")



In [ ]:
# exports links as pickle file
with open("linklist_allsides_news.pickle", "wb") as f:
    pickle.dump(links, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# exports links as csv
with open("linklist_allsides_news.csv", "w") as f:
    for line in links:
        print(line, file=f)

In [ ]:
# functions that assert the existence of
def check_exists_by_xpath(xpath):
    try:
        driver.find_elements(By.XPATH, xpath)[0]
    except:
        return False
    return True

def check_exists_by_class(inp):
    try:
        driver.find_elements(By.CLASS_NAME, inp)[0]
    except:
        return False
    return True

In [ ]:
# load link list from pickle file
with open("linklist_allsides_news.pickle", "rb") as f:
    links = pickle.load(f)

In [ ]:
# list for results
data = []

# retrieve information from articles in list of links
for li in tqdm(links):
    time.sleep(wait_time)
    driver = webdriver.Chrome(service=serv, options=chrome_options)
    print(li)
    # open URL
    driver.get(li)

    # interact with pop-up window
    if check_exists_by_class("css-47sehv"):
        ele = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((driver.find_element(By.CLASS_NAME, "css-47sehv"))))
        ele.click()
    else:
        print("no button")

    # netral title heading
    try:
        heading = driver.find_element(By.TAG_NAME, "h1").text
    except:
        print("no heading found")

    print(heading)

    # date
    try:
        date = driver.find_element(By.CLASS_NAME, "date-display-single").text
    except:
        date = ""

    # tags
    try:
        tags = [a.text for a in driver.find_element(By.CLASS_NAME, "page-tags").find_elements(By.TAG_NAME, "a")]
    except:
        tags = ""


    # define XPATH inforamtion for article divs
    divs = ["/html/body/div[4]/div/div/div/div[4]/div/div/div/div[1]", "/html/body/div[4]/div/div/div/div[4]/div/div/div/div[2]", "/html/body/div[4]/div/div/div/div[4]/div/div/div/div[3]"]

    # access information in article divs
    for d in divs:
        if check_exists_by_xpath(d):
            div = driver.find_elements(By.XPATH, d)[0]

            # check heading element to find out left/center/right. The title contains the bias label that we can retrieve from the text here
            try:
                cat = div.find_element(By.TAG_NAME, "h3").text
            except:
                print("no headline found")

            # retrieve link to original article
            try:
                link = div.find_element(By.TAG_NAME, "a").get_attribute("href")
                print(link)
            except:
                print("no link found")

            # left/center/right are shuffled for each article, some roundups have e.g. only left and right articles.
            # Thus, we have look to look each article seperately
            time.sleep(0.2)
            if "Left" in cat:
                print("left")
                try:
                    left_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading
                except:
                    left_heading = ""
                    print("no headline found")
                try:
                    left_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source
                except:
                    left_source = ""
                    print("no source found")
                try:
                    left_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text body-contents
                except:
                    left_text = ""
                    print("no text found")

                # add the article information
                data.append({"url":link, "date":date, "title": heading, "tags": tags, "heading":left_heading, "source": left_source, "text": left_text, "bias_rating": "left"})

            elif "Right" in cat:
                print("right")
                try:
                    right_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading
                except:
                    right_heading = ""
                    print("no headline found")
                try:
                    right_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source
                except:
                    right_source = ""
                    print("no source found")
                try:
                    right_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text
                except:
                    right_text = ""
                    print("no text found")

                # add the article information
                data.append({"url":link, "date":date, "title": heading, "tags": tags, "heading":right_heading, "source": right_source, "text": right_text, "bias_rating": "right"})

            else:
                print("center")
                try:
                    center_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading
                except:
                    center_heading = ""
                    print("no headline found")
                try:
                    center_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source
                except:
                    center_source = ""
                    print("no source found")
                try:
                    center_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text
                except:
                    center_text = ""
                    print("no text found")

                # add the article information
                data.append({"url":link, "date":date, "title": heading, "tags": tags, "heading":center_heading, "source": center_source, "text": center_text, "bias_rating": "center"})
        else:
            print("div not found")

    driver.close()

    # clear output
    clear_output()
    # added a wait here to assert the scraper runs well
    time.sleep(wait_time)

In [ ]:
## convert data to dataframe
df = pd.DataFrame(data)

# export scraped articles
df.to_csv("allsides_news_complete.csv")